### Globals

In [61]:
import os
import glob
import subprocess
import sys
import random
import numpy as np

rootdir = '/project_freenas/3022017.02/UKB'
qcdir = os.path.join(rootdir, 'qc', 'dMRI')

cluster_log_dir = '/home/preclineu/ramcir/DCCN/TorqueLogs'

cmd_qsub_base = ['/home/preclineu/ramcir/DCCN/Scripts/SubmitToCluster.py',
                 '-length', '102400',
                 '-memory', '2gb',
                 '-logfiledir', cluster_log_dir
                ]

sub_dirs = sorted(glob.glob(os.path.join(rootdir,'subjects','*')))
print('Found', len(sub_dirs), 'subjects to process')

Found 42766 subjects to process


### Sort subjects

In [64]:
sub_ready = []
sub_complete = []

# Control the number of subjects
sdirs = sub_dirs

for s in sdirs:
    subid = os.path.splitext(os.path.basename(s))[0]

    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    quaddir = os.path.join(dmridir, 'data.qc')
    ssepath = os.path.join(dmridir, 'dti_sse.nii.gz')  
    
    try:
        #Check if quad is avalable
        if os.path.exists(quaddir) == False:  
            pass
        else:  
            #Check if sse already exists
            if os.path.exists(ssepath) == True:
                sub_complete.append(s)
            else:
                 sub_ready.append(s)
    except:
        print(subid, ' ignored due to', sys.exc_info()[0], '. Continuing with the next subject.')
        
print('Found', len(sub_no_dmri), 'subjects who are missing dMRI data')
print('Found', len(sub_ready), 'subjects ready to process')
print('Found', len(sub_complete), 'subjects already processed')


Found 19608 subjects who are missing dMRI data
Found 23158 subjects ready to process
Found 0 subjects already processed


### Create sample of random participants

In [65]:
sample_qc_man = random.sample(sub_ready, 1000)
sample_path = os.path.join(qcdir,'sample_qc_man.npy')
np.save(sample_path, sample_qc_man)
sample = np.load(sample_path)


### Run DTIfit on sample

In [67]:
for s in sample[0:]:
    subid = os.path.splitext(os.path.basename(s))[0]
    print('Running DTIfit on subject ' + subid)
    #create paths for all files used in bash command
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    
    #command for extracting b=1000 volumes
    dwiextract_cmd = ['dwiextract --export_grad_fsl ',
                       dmridir + '/dwi_b1000.new_vecs',  
                       dmridir + '/dwi_b1000.new_vals',
                       '-fslgrad ',                           
                       dmridir + '/bvecs ' + dmridir + '/bvals', 
                       '-shell 0.0,1000.0 ' + dmridir + '/data_ud.nii.gz',
                       dmridir + '/dwi_b1000.nii.gz ' + '-force']

    #command for creating mask of b=1000 volumes
    bet_cmd = ['bet ' + dmridir + '/dwi_b1000.nii.gz',
                dmridir + '/dwi_b1000_brain.nii.gz' + ' -f 0.20 -m -R']

    #command for DTIfit - creating dti parameters including sse
    dtifit_cmd = ['dtifit',
                  '-k ' + dmridir + '/dwi_b1000.nii.gz',
                  '-o ' + dmridir + '/dti',
                  '-m ' + dmridir + '/dwi_b1000_brain.nii.gz', 
                  '-r ' + dmridir + '/dwi_b1000.new_vecs', 
                  '-b ' + dmridir + '/dwi_b1000.new_vals',
                  '--sse']

    #Join comands and sumbit process
    proc_cmd = '"%s"' % str(' '.join(dwiextract_cmd) + '&& ' + ' '.join(bet_cmd) + ' && ' + ' '.join(dtifit_cmd))
    cmd_qsub = cmd_qsub_base + ['-name', 'UKB_dtifit_' + subid ,'-command', proc_cmd]   
#     print(proc_cmd)
    subprocess.run(' '.join(cmd_qsub), shell=True)

Running DTIfit on subject 2871677
Running DTIfit on subject 2057788
Running DTIfit on subject 1155176
Running DTIfit on subject 2573695
Running DTIfit on subject 1889381
Running DTIfit on subject 1525366
Running DTIfit on subject 5321423
Running DTIfit on subject 3229317
Running DTIfit on subject 3855493
Running DTIfit on subject 2465838
Running DTIfit on subject 4498976
Running DTIfit on subject 5932738
Running DTIfit on subject 5070954
Running DTIfit on subject 1913668
Running DTIfit on subject 1651029
Running DTIfit on subject 2173301
Running DTIfit on subject 2986987
Running DTIfit on subject 5028992
Running DTIfit on subject 2978633
Running DTIfit on subject 5850487
Running DTIfit on subject 4426602
Running DTIfit on subject 5668740
Running DTIfit on subject 5926083
Running DTIfit on subject 3025554
Running DTIfit on subject 2168059
Running DTIfit on subject 1843275
Running DTIfit on subject 3280181
Running DTIfit on subject 5824338
Running DTIfit on subject 1904153
Running DTIfit

### Run slicesdir
To check the binary mask

In [56]:
visualqc = os.path.join(qcdir, 'visual_qc')
if os.path.exists(visualqc) == False:
    os.mkdir(visualqc)
else: pass

##### b1000 data and mask

In [57]:
string = ""

#this is where the slicesdir will be saved
b1000qc = os.path.join(visualqc, 'b1000')
if os.path.exists(b1000qc) == False:
    os.mkdir(b1000qc)
else: pass
os.chdir(b1000qc)

#append the path to b1000 data and the brain mask for each sub
for s in sample[0:]:
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    string+=str(dmridir + '/dwi_b1000.nii.gz ' + dmridir + '/dwi_b1000_brain_mask.nii.gz' + ' ')

#prep and run the slicesdir command
slices_cmd = 'slicesdir'
prep_cmd = slices_cmd + ' -o ' + string
subprocess.run(prep_cmd, shell = True)

CompletedProcess(args='slicesdir -o /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dwi_b1000.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dwi_b1000.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dwi_b1000.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz ', returncode=0)

##### FA and mask

In [59]:
string = ""

#this is where the slicesdir will be saved
faqc = os.path.join(visualqc, 'fa')
if os.path.exists(faqc) == False:
    os.mkdir(faqc)
else: pass
os.chdir(faqc)

#append the path to FA data and the brain mask for each sub
for s in sample[0:]:
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    string+=str(dmridir + '/dti_FA.nii.gz ' + dmridir + '/dwi_b1000_brain_mask.nii.gz' + ' ')

#prep and run the slicesdir command
slices_cmd = 'slicesdir'
prep_cmd = slices_cmd + ' -o ' + string
subprocess.run(prep_cmd, shell = True)

CompletedProcess(args='slicesdir -o /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dti_FA.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dti_FA.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dti_FA.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz ', returncode=0)

SEE and mask

In [60]:
string = ""

#this is where the slicesdir will be saved
sseqc = os.path.join(visualqc, 'sse')
if os.path.exists(sseqc) == False:
    os.mkdir(sseqc)
else: pass
os.chdir(sseqc)

#append the path to FA data and the brain mask for each sub
for s in sample[0:]:
    dmridir = os.path.join(s, 'dMRI', 'dMRI')
    string+=str(dmridir + '/dti_sse.nii.gz ' + dmridir + '/dwi_b1000_brain_mask.nii.gz' + ' ')

#prep and run the slicesdir command
slices_cmd = 'slicesdir'
prep_cmd = slices_cmd + ' -o ' + string
subprocess.run(prep_cmd, shell = True)

CompletedProcess(args='slicesdir -o /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dti_sse.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1007311/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dti_sse.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000853/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dti_sse.nii.gz /home/preclineu/ramcir/Desktop/Diffusion/subjects/1000432/dMRI/dMRI/dwi_b1000_brain_mask.nii.gz ', returncode=0)